[reference script](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data)

# Import Package and Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

Mounted at /content/drive


In [25]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load your dataset from a CSV file
# Replace 'your_dataset.csv' with the path to your CSV file
# Make sure your CSV file contains features and a target column
# For example, X as features and y as the target column
# Modify this part according to your actual dataset structure
#data = pd.read_csv('/content/drive/MyDrive/Wildfire/Datasets/merged_tp_precip_wind_fmc_all.csv')
data = pd.read_csv("/content/drive/MyDrive/Wildfire/Datasets/merged_tp_precip_wind_fmc_all.csv")
data.dropna(inplace = True)
data.drop(["Unnamed: 0"], axis = 1, inplace = True)
data.drop(["START_DATE"], axis = 1, inplace = True)
data.drop(["END_DATE"], axis = 1, inplace = True)
data.drop(["DATE"], axis = 1, inplace = True)

#data["DATE"] = data["DATE"].astype(float)
#data.drop(["FIRE_SIZE"], axis = 1, inplace = True)
#data['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS'].replace({'A':0 , 'B':1 , 'C':2 ,'D':3 , 'E':4 , 'F':5 , 'G':6 })


In [26]:
data.groupby("FIRE_SIZE_CLASS").count()

,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS,tmax,tmin,tavg,prcp,aws,fmc
FIRE_SIZE_CLASS,,,,,,,,,,
A,93420,93420,93420,93420,93420,93420,93420,93420,93420,93420
B,49237,49237,49237,49237,49237,49237,49237,49237,49237,49237
C,21023,21023,21023,21023,21023,21023,21023,21023,21023,21023
D,10831,10831,10831,10831,10831,10831,10831,10831,10831,10831
E,12014,12014,12014,12014,12014,12014,12014,12014,12014,12014
F,16212,16212,16212,16212,16212,16212,16212,16212,16212,16212
G,15583,15583,15583,15583,15583,15583,15583,15583,15583,15583


In [27]:

def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, 99),
    'B': (99.000000000001, 999),
    'C': (999.000000000001, 99999999999999)
}
  # Creating a new column to store the classifications
classify_ranges(data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')
data.drop(["FIRE_SIZE"], axis = 1, inplace = True)
data.groupby("FIRE_SIZE_CLASS").count()
data['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS'].replace({'A':0 , 'B':1 , 'C':2 ,'D':3 , 'E':4 , 'F':5 , 'G':6 })

In [30]:


# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)  # Replace 'target_column_name' with your actual target column name
y = data['FIRE_SIZE_CLASS']  # Replace 'target_column_name' with your actual target column name
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define your neural network model for 7-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))

Epoch 1/10
5458/5458 [==============================] - 20s 3ms/step - loss: 0.3471 - accuracy: 0.9046 - val_loss: 0.3276 - val_accuracy: 0.9065
Epoch 2/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3265 - accuracy: 0.9060 - val_loss: 0.3212 - val_accuracy: 0.9065
Epoch 3/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3212 - accuracy: 0.9060 - val_loss: 0.3183 - val_accuracy: 0.9063
Epoch 4/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3181 - accuracy: 0.9058 - val_loss: 0.3154 - val_accuracy: 0.9063
Epoch 5/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3162 - accuracy: 0.9060 - val_loss: 0.3148 - val_accuracy: 0.9065
Epoch 6/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3143 - accuracy: 0.9058 - val_loss: 0.3134 - val_accuracy: 0.9065
Epoch 7/10
5458/5458 [==============================] - 14s 3ms/step - loss: 0.3129 - accuracy: 0.9060 - val_loss: 0.3113 - val_ac

In [ ]:
print(accuracy_score(y_test, y_pred_labels))